In [4]:
# Mount the lakehouse oficial structure
import pyfabricops as pf
pf.set_auth_provider("env") 

# set the path to the lakehouse
path = './workspace' 

# Set the workspace and lakehouse
workspace_name = '_pyfabricops-PRD'
workspace_id = pf.resolve_workspace(workspace_name) 
print(workspace_id) 

a6f837e7-7cd8-4ea7-bc5b-2898f3c85704


In [16]:
import pandas as pd

def _flatten_json(data, parent_key='', sep='_'):
    """
    Helper function to flatten nested JSON.

    Args:
        data (dict): JSON to flatten.
        parent_key (str): Parent key (used for recursion).
        sep (str): Separator for flattened keys.

    Returns:
        list[dict]: List of flattened dictionaries.
    """
    items = []
    for k, v in data.items():
        new_key = f'{parent_key}{sep}{k}' if parent_key else k
        if isinstance(v, dict):
            items.extend(_flatten_json(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


def _json_df(data):
    """
    Converts various types of JSON to a DataFrame.

    Args:
        data (dict | list): The JSON to be converted. Can be a simple dictionary,
        a nested dictionary, or a list of dictionaries.

    Returns:
        pd.DataFrame: The resulting DataFrame.
    """
    if not data:
        return None

    if isinstance(data, dict):

        # If it"s a simple dictionary
        if all(not isinstance(v, (dict, list)) for v in data.values()):
            return pd.DataFrame([data])

        # If it"s a dictionary with nested levels
        else:
            flattened_data = _flatten_json(data)
            return pd.DataFrame([flattened_data])

    elif isinstance(data, list):

        # If it"s a list of dictionaries
        if all(isinstance(item, dict) for item in data):
            flattened_list = [_flatten_json(item) for item in data]
            return pd.DataFrame(flattened_list)
        else:
            raise ValueError(
                'The list contains items that are not dictionaries.'
            )

    else:
        raise TypeError(
            'Input type must be a dictionary or a list of dictionaries.'
        )

In [ ]:
# Listing all lakehouses in the workspace
lakehouses = pf.list_lakehouses(workspace=workspace_id) 
lakehouses_df = _json_df(lakehouses) 
display(lakehouses_df)

In [ ]:
# Find first lakehouse with name LakeUnique
lakehouse_name = 'LakeUnique'
for lakehouse in lakehouses:
  if lakehouse['displayName'] == lakehouse_name:
    lakehouse_id = lakehouse['id']
    print(lakehouse_id)
    break

399ff0f0-fa34-4e5a-a481-c4424dfc3c9c


In [24]:
# Export the basic lakehouse to path
pf.export_workspace_config(workspace_id, path)
pf.export_folders(workspace_id, path) 
pf.export_lakehouse(workspace_id, lakehouse_id, path)  

In [54]:
# List the shortcuts of the lakehouse
shortcuts_list = pf.list_shortcuts(workspace_id, lakehouse_id) 
print(shortcuts_list)  

[{'name': 'CurrentDate', 'path': '/Tables', 'target': {'type': 'OneLake', 'oneLake': {'itemId': '56a222c9-2e43-4318-8926-9d20a02c489f', 'path': 'Tables/dbo/CurrentDate', 'workspaceId': 'a6f837e7-7cd8-4ea7-bc5b-2898f3c85704'}}}]


In [55]:
shortcut_dict = shortcuts_list[0]
print(shortcut_dict) 

{'name': 'CurrentDate', 'path': '/Tables', 'target': {'type': 'OneLake', 'oneLake': {'itemId': '56a222c9-2e43-4318-8926-9d20a02c489f', 'path': 'Tables/dbo/CurrentDate', 'workspaceId': 'a6f837e7-7cd8-4ea7-bc5b-2898f3c85704'}}}


In [56]:
shortcut_target = shortcut_dict['target']
print(shortcut_target) 

{'type': 'OneLake', 'oneLake': {'itemId': '56a222c9-2e43-4318-8926-9d20a02c489f', 'path': 'Tables/dbo/CurrentDate', 'workspaceId': 'a6f837e7-7cd8-4ea7-bc5b-2898f3c85704'}}


In [57]:
shortcut_target_type = shortcut_target['type']  
print(shortcut_target_type) 

OneLake


In [58]:
shortcut_target_workspace_id = shortcut_target['oneLake']['workspaceId']
print(shortcut_target_workspace_id) 

a6f837e7-7cd8-4ea7-bc5b-2898f3c85704


In [59]:
shortcut_target_item_id = shortcut_target['oneLake']['itemId'] 
print(shortcut_target_item_id) 

56a222c9-2e43-4318-8926-9d20a02c489f


In [60]:
workspace_items = pf.list_items(shortcut_target_workspace_id) 
for item in workspace_items:
  if item['id'] == shortcut_target_item_id:
    shortcut_target_item_type = item['type']
    print(shortcut_target_item_type)
    break

Warehouse


In [61]:
# Check if the workspace_id is equal shortcut_target_workspace_id then uuid zero
if shortcut_target_workspace_id == workspace_id:
    shortcut_target_workspace_id = "00000000-0000-0000-0000-000000000000"

In [63]:
# Create item type if not exists
if 'itemType' not in shortcut_dict['target']['oneLake']:
    shortcut_dict['target']['oneLake']['artifactType'] = ""
if 'workspaceId' not in shortcut_dict['target']['oneLake']:
    shortcut_dict['target']['oneLake']['workspaceId'] = ""

# Update if exists
shortcut_dict['target']['oneLake']['artifactType'] = shortcut_target_item_type
shortcut_dict['target']['oneLake']['workspaceId'] = shortcut_target_workspace_id

print(shortcut_dict)  

{'name': 'CurrentDate', 'path': '/Tables', 'target': {'type': 'OneLake', 'oneLake': {'itemId': '56a222c9-2e43-4318-8926-9d20a02c489f', 'path': 'Tables/dbo/CurrentDate', 'workspaceId': '00000000-0000-0000-0000-000000000000', 'artifactType': 'Warehouse'}}}


In [64]:
shortcuts_list_new = []
shortcuts_list_new.append(shortcut_dict) 
print(shortcuts_list_new) 

[{'name': 'CurrentDate', 'path': '/Tables', 'target': {'type': 'OneLake', 'oneLake': {'itemId': '56a222c9-2e43-4318-8926-9d20a02c489f', 'path': 'Tables/dbo/CurrentDate', 'workspaceId': '00000000-0000-0000-0000-000000000000', 'artifactType': 'Warehouse'}}}]


In [65]:
# Write the shortcuts to path
with open(f'./workspace/workspace/Storage/LakeUnique.Lakehouse/shortcuts.metadata.json', 'w') as f:
    import json
    json.dump(shortcuts_list_new, f, indent=2) 

In [66]:
# Create dummy lakehouse.metadata.json  
with open(f'./workspace/workspace/Storage/LakeUnique.Lakehouse/lakehouse.metadata.json', 'w') as f:
    import json
    json.dump({}, f, indent=2) 

In [ ]:
# Creating aditional fields in .platform  
with open(f'./workspace/workspace/Storage/LakeUnique.Lakehouse/.platform', 'w') as f:
  platform_content = json.load(f)
print(platform_content)

{'metadata': {'type': 'Lakehouse', 'displayName': 'LakeUnique', 'description': 'The unique lakehouse of the project.'}}


In [70]:
import uuid

if 'config' not in platform_content:
  platform_content['config'] = {}
  
  # Generate a unique ID 
  logical_id = str(uuid.uuid4())

  platform_config = {"version": "2.0", "logicalId": logical_id}
  platform_content['config'] = platform_config


if '$schema' not in platform_content:
    platform_content['$schema'] = ""
    platform_content['$schema'] = "https://developer.microsoft.com/json-schemas/fabric/gitIntegration/platformProperties/2.0.0/schema.json"



In [71]:
# Write reviewed platform file 
with open(f'./workspace/workspace/Storage/LakeUnique.Lakehouse/.platform', 'w') as f:
    json.dump(platform_content, f, indent=2) 